In [3]:
import pandas as pd 

data = pd.read_csv("second_cars_info.csv")#导入数据

#字符串数据转换为日期
data["Boarding_time"].where(data["Boarding_time"]!="未上牌","2030年1月",inplace=True)#使用where替换
data.Boarding_time = pd.to_datetime(data["Boarding_time"],format="%Y年%m月").map(str).str.slice(0,7)
data["Boarding_time"].where(data["Boarding_time"]!="2030-01",pd.NA,inplace=True)#使用where将未上牌数值替换为空
data.sort_values("Boarding_time",inplace=True)#按照时间排序

#字符串数据转换为数值
data["Km"].where(data["Km"]!="百公里内","0.009万公里",inplace=True)#使用where替换
data.Km = data["Km"].apply(lambda x:float(x[:-3])*10000)#里程转换为数值

data["New_price"].where(data["New_price"]!="暂无","0.00万",inplace=True)#使用where替换
data.New_price = data["New_price"].apply(lambda x:float(x[:-1]))#现价转换为数值
data["New_price"].where(data["New_price"]!=0.00,pd.NA,inplace=True)#使用where将现价暂无替换空

def query(key):
    q = []
    li1 = key.split(";")#拆分检索字段
    for i in li1:
        li2 = i.split("=")#取出字段名称和筛选条件
        if li2[0] in ["Boarding_time","Km","Sec_price","New_price"]:
            li3 = li2[-1][1:-1].split(":")
            if len(li3)==1:
                q.append('(data["{}"]=={})'.format(str(li2[0]),li3[0]))
            else:
                if li3[0]=="":
                    q.append('(data["{}"]<{})'.format(str(li2[0]),li3[-1]))
                elif li3[-1]=="":
                    q.append('(data["{}"]>={})'.format(str(li2[0]),li3[0]))
                else:
                    q.append('(data["{}"]>={})'.format(str(li2[0]),li3[0]))
                    q.append('(data["{}"]<{})'.format(str(li2[0]),li3[-1]))
            
        else:
            q.append('(data["{}"].isin({}))'.format(str(li2[0]),li2[-1]))
    q = '&'.join(q)#重新拼接查询条件
    df = data[eval(q)]#查询结果
    df = pd.pivot_table(df,index=["Brand","Name"])#绘制透视表
    return df
if __name__=="__main__":
    quit = ''
    while quit!='q':
        key = str(input("查询条件"))
        try:
            df = query(key)
            print(df)
        except:
            print("没有满足条件的记录，请核对查询条件")
        quit=input("回车(继续)\q(退出)")
#运行程序，输入查询条件：
#例：Brand=["奔驰","大众"];Km=[1000:]---查询Brand为"奔驰"和"大众"并且里程大于1000公里的记录。
#Brand=["奔驰","大众"];Km=[1000:2000]---查询Brand为"奔驰"和"大众"并且里程大于等于1000公里小于2000公里的记录。
#Brand=["奔驰","大众"];Km=[1000:2000];New_price=[100:200]---查询Brand为"奔驰"和"大众"并且里程大于等于1000公里小于2000公里，现价在100-200万之间的记录。

                                      Km  New_price  Sec_price
Brand Name                                                    
大众    POLO 2002款 1.4两厢手动基本型     170000.0      11.27      1.680
      POLO 2002款 1.4两厢手动舒适型     120000.0      10.84      1.050
      POLO 2002款 1.4两厢自动舒适型     136000.0      12.26      1.200
      POLO 2002款 1.6两厢手动舒适型     180000.0      11.94      1.000
      POLO 2003款 1.4三厢手动舒适型     115000.0      12.24      1.400
...                                  ...        ...        ...
奔驰    迈巴赫S级 2015款 400 4MATIC     41950.0     156.09    136.725
      迈巴赫S级 2015款 600            19900.0     313.48    316.100
      迈巴赫S级 2016款 400 4MATIC     55500.0     159.35    139.800
      迈巴赫S级 2017款 S 500 4MATIC    6900.0     238.59    269.000
      迈巴赫S级 2017款 S 600           2000.0     313.48    378.000

[783 rows x 3 columns]
